With GridMethod.jl we can solve real polynomial systems....


In [1]:
import Pkg
Pkg.add("HomotopyContinuation")
Pkg.add("Plots")

   Resolving package versions...
  No Changes to `~/Dropbox/GitHubGalois/GridMethod.jl/Project.toml`
  No Changes to `~/Dropbox/GitHubGalois/GridMethod.jl/Manifest.toml`
   Resolving package versions...
  No Changes to `~/Dropbox/GitHubGalois/GridMethod.jl/Project.toml`
  No Changes to `~/Dropbox/GitHubGalois/GridMethod.jl/Manifest.toml`


In [2]:
import HomotopyContinuation.ModelKit
const HCMK = ModelKit

HomotopyContinuation.ModelKit

In [3]:
using .Iterators
using Test
using LinearAlgebra
using GridMethod.NormsPolynomials
using GridMethod.ConditionNumbers
const CN = ConditionNumbers
using GridMethod.GridModule
using GridMethod.Han
using GridMethod.Coordinates
using GridMethod
using HomotopyContinuation
using Plots

In [4]:
using Plots

function gridGroupByDepth(G::Grid{T, dim}) where {T, dim}
    I = unique(sort(depth.(G)))
    out = Dict(I .=> [empty(G) for _ in I ])
    for g in G
        d = depth(g)
        push!(out[d], g)
    end
    return out
end

xyAxes(G) = eachrow(reduce(hcat, map(coordinates, gridnodes(G))))

@recipe function f(G::Grid{Float64,2};
                   r = 2,
                   R = 20,
                   f_ms = i -> r + R*(.5)^i, # Marker size
                   f_ma = (d, dmin, dmax) -> (d - dmin + 1)/(dmax - dmin + 1), # Mark alpha/opacity
                   f_msw = (d, dmin, dmax) -> r/10 + (1 - f_ma(d, dmin, dmax))/r, # Border width (0 = No border)
                   color = "rgb(238,37,35)"
                   )
    Dict_depth = gridGroupByDepth(G)
    I = sort!(collect(keys(Dict_depth))) # Sort it for the legend
    if !isempty(Dict_depth)
    dmin = first(I)
    dmax = last(I)
    else
    dmin=0
    dmax=0
    end
    # Set default values
    # axis --> ([-1.1,1.1],)
    ticks --> nothing
    xaxis --> false
    yaxis --> false
    lims --> [-2,2]
    aspect_ratio --> 1
    legend --> :outerright
    legendfontsize --> 10
    thickness_scaling --> 1
    shift = dmin -1
    # Plot each depth group
    for d in I
        x, y = xyAxes(Dict_depth[d])
        @series begin
            seriestype := :scatter
            label := "Depth $d"
            # markershape := :circle
            ms := f_ms(d - shift) # Mark size
            mc := color # Mark color
            ma := f_ma(d, dmin, dmax) # Mark alpha/opacity
            msw := f_msw(d, dmin, dmax) # Border/stroke width
            # msw := 0 # No border/stroke
            msc := color # Border color
            msa := f_ma(d, dmin, dmax)/r # Border alpha
            x, y
        end
    end
end

In [5]:
function Exclusion!(
    G::Grid{T, dim},
    extraDepth::UInt,
    degscaling::Vector{T};
    scale::T = one(T),
    nodeFilter=(node)->true,
    _split=splitCoordinate
) where {T, dim}
    gridLock = ReentrantLock()
    Threads.@threads for _ in 1:length(G)
        node = nothing
        lock(gridLock)
        try
            node = popfirst!(G)
        finally
            unlock(gridLock)
        end
        newNodes = []
        if nodeFilter(node)
            newDepth = depth(node)+extraDepth
            cond=condition(node)
            newCoordinates = _split(
                coordinates(node),
                newDepth;
                depth=depth(node),
                scale=scale,
            )
            for coord in newCoordinates
                newNode = LazyGridNode(
                    G,
                    newDepth,
                    coord,
                    cond
                )
                if nodeFilter(newNode)
                    if _ExclusionTest(newNode,degscaling)
                    push!(newNodes,newNode)
                    end
                end
            end
        end
        lock(gridLock)
        try
            append!(G,newNodes)
        finally
            unlock(gridLock)
        end
    end
end

function _ExclusionTest(node::GridNode{T, dim},degscaling::Vector{T}) where {T, dim}
    return 2^depth(node)*maximum(abs.(node.image./degscaling))<=1.1
end

function Inclusion!(
    G::Grid{T, dim},
    extraDepth::UInt,
    p::Real;
    scale::T = one(T),
    nodeFilter=(node)->true,
    _split=splitCoordinate
) where {T, dim}
    gridLock = ReentrantLock()
    Threads.@threads for _ in 1:length(G)
        node = nothing
        lock(gridLock)
        try
            node = popfirst!(G)
        finally
            unlock(gridLock)
        end
        newNodes = []
        if nodeFilter(node)
            newDepth = depth(node)+extraDepth
            cond=condition(node)
            newCoordinates = _split(
                coordinates(node),
                newDepth;
                depth=depth(node),
                scale=scale,
            )
            for coord in newCoordinates
                newNode = LazyGridNode(
                    G,
                    newDepth,
                    coord,
                    cond
                )
                if nodeFilter(newNode)
                    if _InclusionTest(newNode,p)
                    push!(newNodes,newNode)
                    end
                end
            end
        end
        lock(gridLock)
        try
            append!(G,newNodes)
        finally
            unlock(gridLock)
        end
    end
end

function _InclusionTest(node::GridNode{T, dim},p::Real) where {T, dim}
    return norm(node.jacobian\node.image,Inf)*node.condition<=0.25
end

_InclusionTest (generic function with 1 method)

In [6]:
HCMK.@var x,y
polysys=HCMK.System([(x*y-0.1)*(x*y+0.3)];variables=[x,y])

System of length 1
 2 variables: x, y

 (0.3 + x*y)*(-0.1 + x*y)

In [7]:
gridPolySys=GridSystem(polysys,PolyNorm1)
degscaling=convert(Vector{Float64},gridPolySys.degrees)
p=Inf
grid = Grid{Float64, 2}(gridPolySys, [], nothing)
gridHan!(grid,UInt(1);maxDepth=UInt(21))

214.32974222056043

In [8]:
plot(grid)
#savefig("fig0.pdf")

"/Users/tonellicueto/Dropbox/GitHubGalois/GridMethod.jl/notebooks/fig0.pdf"

In [10]:
Exclusion!(grid,UInt(1),degscaling)
plot(grid)
#savefig("fig1.pdf")

"/Users/tonellicueto/Dropbox/GitHubGalois/GridMethod.jl/notebooks/fig1.pdf"

In [11]:
Exclusion!(grid,UInt(1),degscaling)
plot(grid)
#savefig("fig2.pdf")

"/Users/tonellicueto/Dropbox/GitHubGalois/GridMethod.jl/notebooks/fig2.pdf"

In [12]:
Exclusion!(grid,UInt(1),degscaling)
plot(grid)
#savefig("fig3.pdf")

"/Users/tonellicueto/Dropbox/GitHubGalois/GridMethod.jl/notebooks/fig3.pdf"

In [13]:
Inclusion!(grid,UInt(1),p)
plot(grid)
#savefig("fig4.pdf")

"/Users/tonellicueto/Dropbox/GitHubGalois/GridMethod.jl/notebooks/fig4.pdf"